### All TGT

In [ ]:
import ast
from typing import List, Tuple
from google import genai
import re
import time
from util import get_prompt_pairs
def load_prompts_from_file(filename: str) -> List[Tuple[str, str, float]]:
    """Load prompts from text file with format ('neg', 'pos'), Score: x.xxxx"""
    prompts = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Extract the tuple part (handling both quoted and unquoted Score)
            tuple_part = line.split('Score:')[0].strip()
            score_str = line.split('Score:')[1].strip() if 'Score:' in line else '0'
            
            try:
                # Clean the tuple string if needed
                tuple_str = tuple_part.replace('(', '').replace(')', '').split(',')
                neg = tuple_str[0].strip().strip("'")
                pos = tuple_str[1].strip().strip("'")
                score = float(score_str)
                prompts.append((neg, pos, score))
            except (ValueError, IndexError) as e:
                print(f"Skipping malformed line: {line} - Error: {e}")
                continue
                
    return prompts
def filter_prompts(client,prompt_batch: List[Tuple[str, str, float]]) -> List[Tuple[str, str, float]]:
    """Filter prompts using Gemini to remove similar medical concepts"""
    # Create a mapping from prompt text to score for lookup
    prompt_to_score = {(neg, pos): score for neg, pos, score in prompt_batch}
    
    # Prepare the input for Gemini
    prompt_pairs_str = "\n".join([f"('{neg}', '{pos}'), {score:.4f}" 
                                for neg, pos, score in prompt_batch])
    
    # Create the prompt for the LLM
    llm_prompt = f"""The task is to filter textual description pairs of visual discriminative features for tumor detection in histopathology.
    Current prompt pairs:
    {prompt_pairs_str}
    
    Remove pairs that have the same medical concepts but differ only in language variations to make the score high.
    Only provide the output as Python code in the exact format: prompts: list[tuple[negative: str, positive: str], score:int]. Let’s think step by step.
    """



    # response = client.models.generate_content(
    #     model="gemma-3-27b-it",
    #     contents=f"""The task is to filter textual description pairs of visual discriminative features for tumor detection in histopathology. 
    #     Remove pairs that have the same medical concepts but differ only in language variations.
        
    #     Current prompt pairs with scores:
    #     {prompt_pairs_str}
        
    #     Only provide the output as Python code in the exact format: 
    #     prompts = [
    #         ("negative description 1", "positive description 1"), score_1,
    #         ("negative description 2", "positive description 2"), score_2,
    #         ...
    #     ]
    #     Each pair should be followed by a comma and its original score.
    #     """
    # )
    
    # Extract the Python code block from the response
    try:
        # Use the get_prompt_pairs function to handle the LLM interaction
        raw_pairs = get_prompt_pairs(
            prompt=llm_prompt,
            llm_client=client, 
            parse_func=util.extract_and_parse_prompt_list_with_scores 
        )
        
        # Process the parsed pairs
        filtered_pairs = []
        for item in raw_pairs:
            if isinstance(item, tuple) and len(item) == 2:
                # Handle case where we got (prompt, response) pairs
                neg, pos = item
                score = prompt_to_score.get((neg, pos), 0.0)
                filtered_pairs.append((neg, pos, score))
            elif isinstance(item, tuple) and len(item) == 3:
                # Handle case where we got (neg, pos, score) directly
                filtered_pairs.append(item)
            else:
                print(f"Skipping malformed item: {item}")
                
        return filtered_pairs
        
    except Exception as e:
        print(f"Error in get_prompt_pairs: {e}")
        return []
    # try:
    #     print(f"{response.text[-1]}")  # Debug: print the full response text
    #     # Find the prompts list in the response
    #     start_idx = response.text.find('prompts = [')
    #     end_idx = response.text.find(']', start_idx) + 1
    #     code_block = response.text[start_idx:end_idx]
        
    #     # Parse the code block line by line
    #     filtered_pairs = []
    #     lines = code_block.split('\n')
    #     for line in lines:
    #         line = line.strip()
    #         if not line or line.startswith('prompts = [') or line.startswith(']'):
    #             continue
                
    #         # Split into prompt part and score part
    #         parts = line.rsplit(',',2)
    #         prompt_part = parts[0].strip()
    #         score_part = parts[1].replace(',', '').strip()
            
    #         try:
    #             # Parse the prompt tuple
    #             neg, pos = ast.literal_eval(prompt_part)
    #             # Parse the score
    #             score = float(score_part)
    #             filtered_pairs.append((neg, pos, score))
    #         except (ValueError, SyntaxError) as e:
    #             print(f"Skipping malformed line: {line} - Error: {e}")
    #             continue
                
    #     return filtered_pairs
        
    # except Exception as e:
    #     print(f"Error parsing response: {e}")
    #     print(f"Response was: {response.text}")
    #     return []


def save_filtered_prompts(prompts: List[Tuple[str, str, float]], filename: str):
    """Save filtered prompts to a Python file in the specified format"""
    with open(filename, 'w') as f:
        f.write("prompts = [\n")
        for neg, pos, score in prompts:
            # Properly escape quotes
            neg_escaped = neg.replace("'", "\\'")
            pos_escaped = pos.replace("'", "\\'")
            f.write(f"    (\"{neg_escaped}\", \"{pos_escaped}\"), {score:.4f},\n")
        f.write("]\n")

def iterative_filtering(client,input_file: str, output_file: str, batch_size: int = 10) -> None:
    """Perform iterative filtering of prompts in batches"""
    all_prompts = load_prompts_from_file(input_file)
    total_prompts = len(all_prompts)
    print(f"Loaded {total_prompts} prompts from {input_file}")
    
    filtered_results = []
    
    # Process in batches
    for i in range(0, total_prompts, batch_size):
        batch = all_prompts[i:i + batch_size]
        # Combine with previous filtered results if not first batch
        if i > 0:
            batch = filtered_results[:] + batch  # Carry over previous results
        # time.sleep(2)  # Optional: sleep to avoid rate limiting
        filtered_batch = filter_prompts(client,batch)
        if len(filtered_batch)==0:
            print(f"Filtering returned no results for batch starting at index {i} {batch}.")
        filtered_results = filtered_batch
        
        print(f"Processed batch {i//batch_size + 1}: Input {len(batch)} -> Output {len(filtered_batch)}")
        print(f"Current total filtered: {len(filtered_results)}")
    
    # Save the final results
    save_filtered_prompts(filtered_results, output_file)
    print(f"\nFinished filtering. Original: {total_prompts} pairs, Filtered: {len(filtered_results)} pairs")
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    # Example usage
    input_filename = "experiment_results/medical_concepts.txt"  # Your input file
    output_filename = "filtered_prompts.py"  # Output Python file
    
    # Initialize your Gemini client here
    from API_KEY import GEMINI_API_KEY
    import util
    # client = genai.Client(api_key=GEMINI_API_KEY)
    client = util.LLMClient()
    
    iterative_filtering(client,input_filename, output_filename)

/home/e19408/fyp/bioenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/e19408/fyp/bioenv/lib/python3.13/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/e19408/fyp/bioenv/lib/python3.13/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/e19408/fyp/bioenv/lib/python3.13/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from

Loaded 1000 prompts from experiment_results/medical_concepts.txt


### Separate

In [ ]:
import ast
from typing import List, Tuple
from google import genai
import re
import time
from util import get_prompt_pairs
def load_prompts_from_file(filename: str) -> List[Tuple[str, str, float]]:
    """Load prompts from text file with format ('neg', 'pos'), Score: x.xxxx"""
    prompts = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            # Extract the tuple part (handling both quoted and unquoted Score)
            tuple_part = line.split('Score:')[0].strip()
            score_str = line.split('Score:')[1].strip() if 'Score:' in line else '0'
            
            try:
                # Clean the tuple string if needed
                tuple_str = tuple_part.replace('(', '').replace(')', '').split(',')
                neg = tuple_str[0].strip().strip("'")
                pos = tuple_str[1].strip().strip("'")
                score = float(score_str)
                prompts.append((neg, pos, score))
            except (ValueError, IndexError) as e:
                print(f"Skipping malformed line: {line} - Error: {e}")
                continue
                
    return prompts
def filter_prompts(client, existing_pairs: List[Tuple[str, str, float]], new_batch: List[Tuple[str, str, float]]) -> List[Tuple[str, str, float]]:
    """Filter prompts using Gemini to remove similar medical concepts"""
    # Create mappings for both existing and new pairs
    existing_to_score = {(neg, pos): score for neg, pos, score in existing_pairs}
    new_to_score = {(neg, pos): score for neg, pos, score in new_batch}
    
    # Prepare the strings for display
    existing_pairs_str = "\n".join([f"('{neg}', '{pos}'), {score:.4f}" 
                                 for neg, pos, score in existing_pairs]) if existing_pairs else "None"
    
    new_pairs_str = "\n".join([f"('{neg}', '{pos}'), {score:.4f}" 
                             for neg, pos, score in new_batch])
    
    # Create the prompt for the LLM
    llm_prompt = f"""The task is to filter textual description pairs of visual discriminative features for tumor detection in histopathology. 
    Remove pairs from the NEW BATCH that have the same medical concepts as either:
    1. Pairs in the EXISTING DISTINCT CONCEPTS list, OR
    2. Other pairs in the NEW BATCH
    
    EXISTING DISTINCT CONCEPTS (already filtered):
    {existing_pairs_str}
    
    NEW BATCH TO FILTER:
    {new_pairs_str}
    
    Only provide the output as Python code in the exact format: 
    prompts = [
        ("negative description 1", "positive description 1"), score_1,
        ("negative description 2", "positive description 2"), score_2,
        ...
    ]
    Each pair should be followed by a comma and its original score. Think step by step.
    """
    
    try:
        # Use the get_prompt_pairs function to handle the LLM interaction
        raw_pairs = get_prompt_pairs(
            prompt=llm_prompt,
            llm_client=client, 
            parse_func=util.extract_and_parse_prompt_list_with_scores 
        )
        print(f"Raw pairs from LLM: {raw_pairs}")  # Debug: print the raw pairs
        # Process the parsed pairs
        filtered_pairs = []
        for item in raw_pairs:
            if isinstance(item, tuple) and len(item) == 2:
                # Handle case where we got (prompt, response) pairs
                neg, pos = item
                score = new_to_score.get((neg, pos), 0.0)
                filtered_pairs.append((neg, pos, score))
            elif isinstance(item, tuple) and len(item) == 3:
                # Handle case where we got (neg, pos, score) directly
                filtered_pairs.append(item)
            else:
                print(f"Skipping malformed item: {item}")
                
        return filtered_pairs
        
    except Exception as e:
        print(f"Error in get_prompt_pairs: {e}")
        return []
    

def save_filtered_prompts(prompts: List[Tuple[str, str, float]], filename: str):
    """Save filtered prompts to a Python file in the specified format"""
    with open(filename, 'w') as f:
        f.write("prompts = [\n")
        for neg, pos, score in prompts:
            # Properly escape quotes
            neg_escaped = neg.replace("'", "\\'")
            pos_escaped = pos.replace("'", "\\'")
            f.write(f"    (\"{neg_escaped}\", \"{pos_escaped}\"), {score:.4f},\n")
        f.write("]\n")

def iterative_filtering(client, input_file: str, output_file: str, batch_size: int = 10) -> None:
    """Perform iterative filtering of prompts in batches"""
    all_prompts = load_prompts_from_file(input_file)
    total_prompts = len(all_prompts)
    print(f"Loaded {total_prompts} prompts from {input_file}")
    
    distinct_pairs = []
    
    # Process in batches
    for i in range(0, total_prompts, batch_size):
        new_batch = all_prompts[i:i + batch_size]
        
        # Filter the new batch against existing distinct pairs
        filtered_batch = filter_prompts(client, distinct_pairs, new_batch)
        
        if len(filtered_batch) == 0:
            print(f"Filtering returned no results for batch starting at index {i} {new_batch}.")
        else:
            # Add the filtered results to our distinct pairs
            distinct_pairs = filtered_batch
        
        print(f"Processed batch {i//batch_size + 1}:")
        print(f"  New batch size: {len(new_batch)}")
        print(f"  After filtering: {len(filtered_batch)}")
        print(f"  Total distinct pairs: {len(distinct_pairs)}")
    
    # Save the final results
    save_filtered_prompts(distinct_pairs, output_file)
    print(f"\nFinished filtering. Original: {total_prompts} pairs, Filtered: {len(distinct_pairs)} pairs")
    print(f"Results saved to {output_file}")

if __name__ == "__main__":
    # Example usage
    input_filename = "experiment_results/medical_concepts.txt"  # Your input file
    output_filename = "filtered_prompts.py"  # Output Python file
    
    # Initialize your Gemini client here
    from API_KEY import GEMINI_API_KEY
    import util
    # client = genai.Client(api_key=GEMINI_API_KEY)
    client = util.LLMClient()
    
    iterative_filtering(client,input_filename, output_filename)

Loaded 1000 prompts from experiment_results/medical_concepts.txt
Loaded 10 prompt-pairs.
First pair: ('No atypical cells infiltrating surrounding tissues', 'Atypical cells infiltrating surrounding tissues and disrupting normal structures', 0.9013)
Raw pairs from LLM: [('No atypical cells infiltrating surrounding tissues', 'Atypical cells infiltrating surrounding tissues and disrupting normal structures', 0.9013), ('No significant atypia in the surrounding lymphocytes', 'Significant atypia observed in lymphocytes adjacent to tumor nests', 0.8997), ('No evidence of fibrosis', 'Prominent stromal fibrosis surrounding tumor nests', 0.8994), ('Normal follicular architecture is preserved', 'Disrupted follicular architecture with loss of polarity', 0.894), ('No prominent nucleoli are observed in lymphocytes', 'Cells exhibit large', 0.8935), ('No giant cells or multinucleated cells are seen', 'Presence of multinucleated giant cells', 0.8884), ('No plasmacytoid differentiation is observed', 'Pla